# 八股

## 1. 一条SQL查询语句是如何执行的？

1. 连接器: 连接器负责和客户端(数据库)进行创建连接、获取权限、维持和管理链接.
2. 缓存查取: 当 MySQL 接收到一个查询请求时, 首先检查查询是否在查询缓存中, 如果存在, 可能是以key-value的形式存放的. 如果命中, 立即返回结果.
3. 分析器: 分析器对 SQL 查询进行语法解析和语义分析. 分析器会根据查询的 SQL 语句, 确定表的读取顺序, 确定表的连接类型, 确定表的查询条件等.
4. 优化器: 优化器会根据 MySQL 版本, 查询的 SQL 语句, 表的结构等, 选择最好的执行计划. 例如, 可能的查询方式, 可能的 join 顺序等.
5. 执行器: 分析器只要要做什么, 优化器知道要怎么做, 最后就是交给执行器, 执行语句.

## 2. 事务的四大特性有哪些?

1. 原子性: 一个事务的处理, 只有全部完成和不完成则回滚, 不存在部分完成情况.
2. 一致性: 在执行事务的前后, 数据库从前一个一致性状态转换成新的一致性状态.
3. 隔离性: 多个事务的执行是互不相干的
4. 持久性: 一个事务一旦提交, 就是永久存储, 即使数据库崩溃也不会丢失.

## 3. 数据库的事务隔离级别有哪些?

### 1. 读未提交（Read Uncommitted）
想象有两个用户，分别操作一个银行账户。

- 事务A：开始转账操作，从账户中扣除100元。
- 事务B：在事务A还未提交时，读取了账户余额。

假如原余额是500元，事务A正在扣除100元，尚未提交，所以账户余额显示为400元。但实际上，这100元的扣款并未确定，事务A可能会回滚，余额仍应为500元。事务B读取到了事务A尚未提交的数据，这种情况叫“脏读”。

### 2. 读已提交（Read Committed）
在这种隔离级别下，事务只能读取已经提交的数据。

- 事务A：开始转账操作，扣除100元，并在操作完成后提交。
- 事务B：只有在事务A提交之后，才读取到账户余额的变化。

假设事务A成功扣除100元并提交，账户余额变为400元，事务B这时读取余额就是400元，不会读取到未提交的数据。因此，避免了“脏读”，但如果事务B在事务A提交之前读取了一次余额（500元），然后事务A提交后再读取一次余额（400元），就会发现数据不一致，这种情况叫“不可重复读”。

### 3. 可重复读（Repeatable Read）
在“可重复读”隔离级别下，事务期间多次读取同一数据会得到一致的结果。

- 事务A：开始转账操作，扣除100元，但在事务A提交前，事务B多次读取余额。
- 事务B：在事务A的操作期间，读取余额，无论多少次都只能看到事务开始时的余额（500元）。

![image.png](./images/image_可重复读.png)

因此，事务B在事务A提交前，多次读取都保持一致，解决了“不可重复读”的问题。但如果事务A在过程中新增了一条数据，比如新增一笔100元的存款，则事务B读取余额列表时可能看到新增的数据，这种现象叫“幻读”。

### 4. 序列化（Serializable）
在最高的隔离级别下，事务是完全串行化的，彼此之间没有并发操作。

- 事务A：转账扣除100元。
- 事务B：在事务A执行完并提交前，事务B无法执行任何读取操作。

这样，不会出现“脏读”、“不可重复读”或“幻读”，因为事务B只有在事务A完全执行完并提交后才能读取数据。

## 4. MySQL 的执行引擎有哪些?

MySQL的执行引擎（即存储引擎）是负责执行查询和存储数据的不同模块。MySQL 支持多种存储引擎，每种引擎在性能、并发性、事务处理等方面有不同的特性，适用于不同的应用需求。下面是主要的存储引擎的简要说明：

1. **MyISAM**  
   - **特性**：MySQL 5.5.5 之前的默认存储引擎，但**不支持事务**，也不支持**表锁**和**外键**。MyISAM 适合读操作多、并发量较小、数据安全性要求不高的场景，因为它只提供基本的数据库功能。
   - **应用场景**：适用于以数据查询为主、更新较少的应用，比如小型博客系统、个人网站等。

2. **InnoDB**  
   - **特性**：MySQL 5.5.5 之后的默认存储引擎，支持**事务**，并满足 ACID（原子性、一致性、隔离性、持久性）特性，还支持**行锁**和**外键**，这些特性使得 InnoDB 更适合需要高并发、高安全性以及严格事务控制的场景。
   - **应用场景**：适用于对数据安全和一致性有严格要求的业务场景，如金融系统、电商平台等，尤其适合高并发写入需求。

3. **Memory**  
   - **特性**：Memory 存储引擎将数据存储在内存中，因此数据读取和写入速度非常快。但一旦数据库重启或关闭，数据会被清空。因此，Memory 引擎通常用于缓存数据，而不是存储持久数据。
   - **应用场景**：适合用于临时数据的高频访问，通常用于缓存，比如会话管理、临时表数据存储等。

**选择存储引擎时需考虑的因素：**
- **事务支持**：若需要事务管理，应选择 InnoDB。
- **数据持久性**：需要持久保存数据的选择 InnoDB 或 MyISAM；对持久性无要求的可以选择 Memory。
- **并发性**：InnoDB 支持行级锁，更适合高并发。

## 5. MySQL为什么使用B+树来作索引

B+ 树和 B 树是两种常用于数据库和文件系统的树形结构，它们都可以快速查找数据，但在结构和性能上有一些关键区别：

### 1. **单点查询效率**
   - **B 树**：在 B 树中，每个节点既包含索引，也包含数据记录，这意味着在进行单个数据查找时，如果数据刚好在非叶子节点，那么可以在较少的步骤内找到，时间复杂度理论上可以是 \(O(1)\)。然而，这种方式的缺点是，B 树的查询性能不稳定：有时查询会较快地找到数据，有时需要访问到叶子节点才能找到，造成查询波动。
   - **B+ 树**：在 B+ 树中，**非叶子节点只存储索引**，而所有的数据都存放在叶子节点。这种结构让 B+ 树通常更“矮胖”，因为非叶子节点可以容纳更多索引。因此，虽然在单点查询上 B+ 树的平均时间复杂度略高，但整体上减少了磁盘 I/O 的次数，因为树的高度较小。

### 2. **插入和删除效率**
   - **B 树**：B 树的节点没有冗余数据，因此删除节点时如果需要重新组织树形结构，就会更复杂；插入时，可能涉及更多节点的重新调整和拆分，效率较低。
   - **B+ 树**：B+ 树的叶子节点之间有冗余节点，它们形成链表结构。这样，删除叶子节点时可以直接操作叶子层而无需调整非叶子节点。插入数据时，如果某节点饱和，则只需调整一条路径的节点，效率更高。

### 3. **范围查询效率**
   - **B 树**：B 树没有将叶子节点串联起来的链表结构，因此在范围查询时需要通过树的遍历完成，这通常涉及多个节点，增加了磁盘 I/O 次数，查询速度较慢。
   - **B+ 树**：B+ 树的叶子节点通过链表相连，这使得范围查询可以沿着链表快速读取。数据库系统中常用 B+ 树，正是因为它更适合需要大量范围查询的应用场景。

### **应用场景**
   - **B 树**：适合主要进行单个数据索引查询的场景，例如 MongoDB。
   - **B+ 树**：由于范围查询效率更高，适用于数据库等场景。

## 6. 说一下索引失效的场景？

索引失效意味着查询操作不能有效利用索引进行数据检索，从而导致性能下降，下面一些场景会发生索引失效。

1. 使用OR条件：当使用OR连接多个条件，并且每个条件用到不同的索引列时，索引可能不会被使用。
2. 使用非等值查询：当使用!=或<>操作符时，索引可能不会被使用，特别是当非等值条件在WHERE子句的开始部分时。
3. 对列进行类型转换： 如果在查询中对列进行类型转换，例如将字符列转换为数字或日期，索引可能会失效。
4. 使用LIKE语句：以通配符%开头的LIKE查询会导致索引失效。
5. 函数或表达式：在列上使用函数或表达式作为查询条件，通常会导致索引失效。
6. 表连接中的列类型不匹配： 如果在连接操作中涉及的两个表的列类型不匹配，索引可能会失效。例如，一个表的列是整数，另一个表的列是字符，连接时可能会导致索引失效。

## 7. 什么是慢查询? 原因是什么? 可以怎么优化?

慢查询是指在数据库中进行数据查询时, 查找时间超时, 这就叫慢查询.

**原因**

1. 所要查找的逻辑十分繁琐, 包含多个表之间的关系以及子查询, 导致运行超时
2. 所要查找的数据量很大, 所以, 即使查询逻辑简单也需要耗费很多时间
3. 数据库同时执行多个查找请求, CPU占用率高, 可能会导致超时
4. 数据库建立的数据结构差, 没有高精准的标识, 导致需要遍历整个表才能找到, 导致超时
5. 当多个查询同时访问同一个资源时, 可能会因为互斥原则导致超时

**优化**

1. 简化sql逻辑, 将复杂的查找优化, 明确简化的逻辑
2. 建立合理的索引, 只有在查询需要的数据时才建立索引, 避免建立不必要的索引
3. 合理使用limit, 避免返回大量的数据, 特别是返回大量的记录

![image.png](./images/image_limit1.png)
![image-2.png](./images/image_limit2.png)

4. 并行查询, 如果可以, 就让多个查询同时执行, 这可以大大提高查询速度
5. 合理使用explain, 分析sql的执行计划, 找到慢查询的原因, 然后对sql进行修改, 让查询更高效

## 8. undo log、redo log、binlog 有什么用？

* undo log是Innodb存储引擎层生成的日志，实现了事务中的原子性，主要用于事务回滚和MVCC。
* redo log是物理日志，记录了某个数据页做了什么修改，每当执行一个事务就会产生一条或者多条物理日志。
* binlog (归档日志）是Server 层生成的日志，主要用于数据备份和主从复制。

## 9. MySQL和Redis的区别是什么?

**存储方式**

MySQL: 是一种关系数据库, 使用表来存储数据

Redis: 是一种键-值存储数据库, 键-值对存储的数据可以是字符串、数字、列表、集合、有序集合等

**数据类型**

MySQL: MySQL有多种数据类型, 如整型、字符型、布尔型等

Redis: Redis支持**字符串类型, Hash（哈希），List（列表），Set（集合）及 Zset(sorted set：有序集合)**。

**数据持久化**

MySQL: 直接存储在磁盘上实现持久化存储

Redis: Redis可以将数据存储在内存中, 也可以将数据存储在硬盘中, 实现持久化存储

**语法**

MySQL: MySQL使用 SQL 语言

Redis: Redis使用命令来操作数据

**应用场景**
Redis: 以高性能、低延迟为目标, 适合多读少写的应用场景

MySQL: 适用于对数据进行查询、事务处理、用于大规模数据的场景.

**总结**

Redis 更适合处理高速、高并发的数据访问，以及需要复杂数据结构和功能的场景，在实际应用中，很多系统会同时使用 MySQL 和 Redis。

## 10. Redis有什么优缺点? 为什么用Redis查询会比较快

**Redis有什么优缺点?**

**优点:**

* 存储在内存中, 读取迅速方便, 常用作缓存存储.
* Redis支持多种数据结构, 如字符串、哈希表、列表、队列等.
* Redis提供分部式服务, 有很好的延展性和可用性.

**缺点:**

* 因为Redis是将数据存储在内存上, 所以, 存储数据的空间受内存的限制, 不适合存储大量的数据, 同时, 内存扩大成本高

**为什么用Redis查询会比较快?**

* 使用内存存储数据, 节省在磁盘上的多次I/O操作
* 使用高效率的数据结构, 如列表, 哈希表等等
* 单线程查询, 避免了多线程之间的互斥等待, 避免了死锁现象
* I/O多路复用, 同时监听多个Socket, 根据Socket上的事件来选择对应的事件处理器进行处理。

## 11. Redis常见的五种数据类型有哪些?

### 1. 字符串 (String)
**字符串**类型是最简单的，类似于键值对存储的结构。

**例子**：假设你在存储用户的姓名或账户余额。
```plaintext
SET user:1001:name "Alice"          // 设置用户1001的姓名为Alice
SET user:1001:balance 5000           // 设置用户1001的账户余额为5000
```

### 2. 哈希表 (Hash)
**哈希表**适合用来存储对象，例如用户的信息（多个字段的数据）。

**例子**：假设你有一个用户对象，需要存储用户的姓名、年龄和邮箱。
```plaintext
HSET user:1001 name "Alice"          // 设置用户1001的姓名为Alice
HSET user:1001 age 25                // 设置用户1001的年龄为25
HSET user:1001 email "alice@example.com"  // 设置用户1001的邮箱
```

### 3. 列表 (List)
**列表**用于存储多个有序元素，可以用作消息队列或任务队列。

**例子**：假设你有一个消息队列，存储需要处理的任务。
```plaintext
LPUSH task:queue "task1"             // 将任务1添加到队列
LPUSH task:queue "task2"             // 将任务2添加到队列
RPOP task:queue                      // 取出并处理最早进入的任务（“task1”）
```

### 4. 集合 (Set)
**集合**存储唯一的元素，适合用于去重场景，元素是无序的。

**例子**：假设你需要存储每个访问特定页面的唯一用户ID。
```plaintext
SADD page:1001:visitors 1001         // 用户1001访问页面1001
SADD page:1001:visitors 1002         // 用户1002访问页面1001
SADD page:1001:visitors 1001         // 用户1001再次访问，但不会重复存储
```

### 5. 有序集合 (ZSet)
**有序集合**是一个集合，每个元素都关联一个分数，可以根据分数进行排序。

**例子**：假设你需要一个排行榜，按分数存储用户。
```plaintext
ZADD leaderboard 1000 "Alice"        // 用户Alice的分数为1000
ZADD leaderboard 1500 "Bob"          // 用户Bob的分数为1500
ZREVRANGE leaderboard 0 1 WITHSCORES // 查询前两名用户
```

### 6. 位图 (BitMap)
**位图**可以按位操作，适合用于标记某些状态。

**例子**：假设你需要记录某个月每天用户是否签到（0表示未签到，1表示已签到）。
```plaintext
SETBIT user:1001:sign 1 1            // 用户1001在1号签到
SETBIT user:1001:sign 2 1            // 用户1001在2号签到
GETBIT user:1001:sign 1              // 查询用户1001在1号是否签到，返回1
```

### 7. HyperLogLog
**HyperLogLog**用于大规模去重和基数估计，适合用于快速统计大量数据中的唯一元素数。

**例子**：假设你想统计一天内访问网站的独立用户数。
```plaintext
PFADD site:unique_visitors 1001      // 用户1001访问
PFADD site:unique_visitors 1002      // 用户1002访问
PFADD site:unique_visitors 1001      // 用户1001再次访问，不会重复计数
PFCOUNT site:unique_visitors         // 返回独立用户数
```

### 8. 地理信息 (GEO)
**GEO**用于存储地理位置坐标，适合用于地点、商店等位置的距离计算。

**例子**：假设你有不同商店的地理位置，需要查找某个地点附近的商店。
```plaintext
GEOADD stores 13.361389 38.115556 "Palermo"  // 添加商店Palermo的经纬度
GEOADD stores 15.087269 37.502669 "Catania"  // 添加商店Catania的经纬度
GEORADIUS stores 15 37 100 km                // 查询距离某位置100公里内的商店
```

### 9. 消息流 (Stream)
**Stream**是专门用于消息队列的，可以存储消息日志和事件流。

**例子**：假设你需要存储系统事件的日志。
```plaintext
XADD mystream * sensor-id 1 temperature 22.5  // 添加一条日志消息
XREAD COUNT 2 STREAMS mystream 0              // 读取日志流中的前两条消息
```

## 12. Redis是单线程的还是多线程的, 为什么?

单线程的.

**原因**

* 简化模型: 简化了并行开发, 避免了同步互斥等问题
* 保证原子性: 因为是单线程执行程序, 不会被其他线程打断, 实现了原子性
* 顺序执行: 单线程保证了线程执行的顺序
* 性能优化: Redis的的大多数操作在内存进行, 使用单线程不用进行繁琐的加锁解锁等复杂操作

同时, 因为是在内存进行操作, 操作执行迅速, 所以Redis的吞吐量依旧很大.在6.0版本之后, Redis也支持了多线程, 用于缓解网络I/O操作, 充分利用CPU资源

## 13. 说一说Redis持久化机制有哪些

* AOF 日志：每执行一条写操作命令，就把该命令以追加的方式写入到一个文件里；
* RDB 快照：将某一时刻的内存数据，以二进制的方式写入磁盘；
* 混合持久化方式：Redis 4.0 新增的方式，集成了 AOF 和 RBD 的优点；